<a href="https://colab.research.google.com/github/majesticio/live-sound-classifier/blob/main/IoT_live_sound_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import IPython.display as ipd
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd 
import wave
import struct
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import train_test_split 
from tensorflow.keras.utils import to_categorical
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 
from playsound import playsound
import os
import librosa
import librosa.display
from pydub import AudioSegment
import numpy as np
import soundfile as sfile
import math
import matplotlib.pyplot as plt
from scipy.io import wavfile
import subprocess
import audioop
import os
import datetime
import pickle

In [ ]:
#Declare path for pickled variables X, y
pickledX = '/home/samus/models/variables/pickledX.x'
pickledY = '/home/samus/models/variables/pickledY.y'

In [ ]:
#Assign X and y to thier respective pickles
X = pickle.load(open(pickledX, 'rb'))
y = pickle.load(open(pickledY, 'rb'))

In [ ]:
#All of the metadata is defined from X and y(!)
# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

# split the dataset 

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

In [ ]:
#Question for Nevin: do I need checkpointer, model.fit? Do I need to compile?
# num_epochs = 128
# num_batch_size = 64
# checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
#                                verbose=1, save_best_only=True)

In [ ]:
# %store

Load a pickeled model.

In [ ]:
# load the model from pickled model path
pickledModelPath = '/home/samus/models/pickeled_model.sav'
model = pickle.load(open(pickledModelPath, 'rb'))

2022-01-26 08:58:44.756376: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-26 08:58:44.793591: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-26 08:58:44.793855: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-26 08:58:44.794486: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
202

Load a pre-trained model.

Very Important

In [ ]:
#Must Declare
num_rows = 40
num_columns = 174
num_channels = 1

Compile the model.

In [ ]:
#Do I need to compile?
model.summary()

In [ ]:
#Define a function that will create a spectrogram for each audio sample. 
#This function uses librosa to standardize the data.

max_pad_len = 174 #original
# max_pad_len = 255

def extract_features(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        # print(sample_rate)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        # print(mfccs)
        pad_width = max_pad_len - mfccs.shape[1]
        # print(mfccs.shape[1])
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)),
                       mode='constant')
        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
     
    return mfccs


In [ ]:
def print_prediction(file_name):
    prediction_feature = extract_features(file_name)
    # print("Printing Type: ")
    # type(prediction_feature)
    # print("Printing Feature")
    # print(prediction_feature)
    prediction_feature = prediction_feature.reshape(1, num_rows, num_columns, num_channels)#very important 

    predicted_vector = model.predict(prediction_feature) 
    # np.argmax(model.predict(x), axis=-1)
    
    predicted_class = np.argmax(predicted_vector,axis=1)
    labels = ["air_conditioner", "car_horn", "children_playing", "dog_bark", "drilling", "engine_idling",
              "glass_breaking", "gun_shot", "jackhammer", "siren", "street_music"]
    print("The predicted class is:", labels[predicted_class[0]], '\n') 

    predicted_proba_vector = model.predict(prediction_feature) 
    predicted_proba = predicted_proba_vector[0]
    for i in range(len(predicted_proba)): 
        category = le.inverse_transform(np.array([i]))
        print(category[0], "\t\t : ", format(predicted_proba[i], '.32f') )

In [ ]:
def checkVolumePredict(filename, sensitivity):
# sensitivity = -40
  audio=AudioSegment.from_mp3(filename)
  signal, sr = sfile.read(filename)
  samples=audio.get_array_of_samples()
  samples_sf=0
  try:
      samples_sf = signal[:, 0]  # use the first channel for dual
  except:
      samples_sf=signal  # for mono


  def convert_to_decibel(arr):

    ref = 1
    if arr !=0:
      return 20 * np.log10(abs(arr) / ref)
          
    else:
      return -60
  data=[convert_to_decibel(i) for i in samples_sf]
  highest = max(data)
  print ("highest reading:",
        highest)
  if highest > sensitivity:
    print_prediction(filename)
    print(highest)
  else:
    subprocess.run(["rm" , filename])
    print('too quiet', highest)

In [ ]:
!pwd

/home/samus


In [ ]:
#Class: glass breaking
filename = '/home/samus/DataCollection/fucked/legit.wav'
print_prediction(filename)

The predicted class is: glass_breaking 

air_conditioner 		 :  0.00000000000603473737995074976936
car_horn 		 :  0.00000000000020534341913479303710
children_playing 		 :  0.00000262768185166351031512022018
dog_bark 		 :  0.00137719768099486827850341796875
drilling 		 :  0.00000000043884143541284004186309
engine_idling 		 :  0.00000000000022233091561373041767
glass_breaking 		 :  0.99861204624176025390625000000000
gun_shot 		 :  0.00000806664411356905475258827209
jackhammer 		 :  0.00000000000002894562302520331531
siren 		 :  0.00000000000021380457354645138190
street_music 		 :  0.00000000000000062228683662311985


In [ ]:
# while (True):
samples_path = '/home/samus/Music/samples/'
# samples_folder = '/content/drive/MyDrive/samples/'
timestamp = datetime.datetime.now().strftime('%Y_%m_%d-%H_%M_%S')
# filename = samples_folder + timestamp + ".wav"
filename = samples_path + timestamp + ".wav"
print(timestamp)

#hw id will change on different hardware
cmd = 'arecord -D hw:3,0 -t wav -f S16_LE -r 44100 -c 2 -d 3 ' + filename 

os.system(cmd) #records a three second sample

checkVolumePredict(filename, -40); #Determines decibel threshold
# playsound(filename)
# checkVolume(filename)

timestamp = datetime.datetime.now().strftime('%Y_%m_%d-%H_%M_%S')
print(timestamp)
